### Some related Functions

In [1]:
import os
from collections import Counter
#read each sdf file's content
def readsdf(filename):
    with open (filename,"r") as rf:
        content = rf.readlines()
        rf.close()
    return content

#obtained compound's name and its comformer matching rmsd
def writerecord(content):
    compoundname = []  #collect compound's name
    recordstring = ""  #record the compound's name and matching comformer's rmsd value
    linenumber = len(content)
    for lineindex in range(linenumber):
        line = content[lineindex].replace("\n","")
        if line[-3:] == "sdf":
            recordstring += line
            recordstring += ","
            compoundname.append(line)
        if line == "> <rmsd>":
            recordstring += content[lineindex+1].replace("\n","")
            recordstring += "\n"
    return recordstring,compoundname 

#same comformers of a compound present in the downloaded sdf file
#hence，remove the repeating comformers by using de_weight function
def de_weight(compoundname,string):
    compoundname = set(compoundname) #de-weight
    recordlist = string.split("\n") 
    string_ = ""
    string__ = ""
    for name in compoundname:
        rmsdlist = []
        for data in recordlist:
            data_ = data.split(",")
            if data_[0] == name:
                rmsdlist.append(data_[1])
        rmsdlist = set(rmsdlist)  #delete the same comformer for this molecule
        number = str(len(rmsdlist))
        string__ += name+","+number+"\n"
        for rmsd in rmsdlist:
            rmsd_ = name+","+rmsd+"\n"
            string_ += rmsd_
    return string_,string__

#export the result
def writefile(filename,string):
    with open(filename,"w") as wf:
        wf.write("compound name,rmsd\n")
        wf.write(string)
        wf.close()
    return None

def writefile2(filename,string):
    with open(filename,"w+") as wf:
        wf.write("decoy name,hit number\n")
        wf.write(string)
        wf.close()    
    return None

### The number of conformer hit in one frame was counted

In [17]:
folder = ".\\output\\PPYhit\\"
print("The sdf files gotten form Pharmit were stored in ["+folder+"]")

The sdf files gotten form Pharmit were stored in [.\output\PPYhit\]


In [18]:
files = os.listdir(folder)
for file in files:
    if file[-3:] != "sdf":
        files.remove(file)

print("The number of pharmacophore frames matched in Pharmit were "+str(len(files)))

The number of pharmacophore frames matched in Pharmit were 7


In [19]:
str3 = "" #collect data contained all sdf-files
for file in files: 
    filename = folder + file
    content = readsdf(filename)
    string,compoundname = writerecord(content)
    str1,str2 = de_weight(compoundname,string)
    str3 += str2 + "\n"

data = str3.split("\n\n")
data.remove("")
print("The number of frames is ",len(data))
del str1,str2,str3

The number of frames is  7


In [20]:
data[0]  #check the data is not blank

'PPY.sdf,1'


### Collect compound name come from origin dataset 

In [21]:
def getcname(filename):
    with open(filename,"r",encoding="utf8") as rf:
        contentlist = rf.readlines()
        rf.close()
    cnames = []
    for line in contentlist:
        line = line.replace("\n","")
        if line[-3:] == "sdf":
            cnames.append(line.replace(".sdf",""))
    cfrequence = Counter(cnames)
    cnames = list(set(cnames)) #de-weight
    return cnames,cfrequence


In [22]:
infilename = ".\\output\\PPY-2.sdf"
print("Note! the sdf send to Pharmit as sdf.gz were ["+infilename+"]")
#make sure the input filename is correct

Note! the sdf send to Pharmit as sdf.gz were [.\output\PPY-2.sdf]


In [23]:
cnames,cfrequence = getcname(infilename) 
#variable cfrequence is a dictionary

print("The origin dataset contained compound number are ",\
      len(list(cfrequence.values())))
print("="*12)
print("This compound names are ",cnames)
print("="*12)
print("Each compound included comformers are ",list(cfrequence.values()))

The origin dataset contained compound number are  1
This compound names are  ['PPY']
Each compound included comformers are  [99]


### Modify result data style 

In [24]:
clist,report= cnames,""

for framedata in data: # string
    framedatalist = framedata.split("\n")
    for c in clist:
        checkpoint = 0
        for peptidedata in framedatalist: #each compound in a sdf- file
            peptidedatalist = peptidedata.split(",")
            if peptidedatalist[0].replace(".sdf","") == c: # if match result in clist variable
                report += c+","+"1\n"
                checkpoint = 1
                break
        if checkpoint != 1:   # if nothing match in clist
            report += c+","+ "0\n"
        if checkpoint == 1:
            checkpoint = 0

datas = report.split("\n")
outputdata,currentline = [],""
for c in clist:
    currentline += c+","
    for data in datas:
        data = data.split(",")
        if data[0] == c:
            currentline += data[1]+","
    currentline += "\n"
    outputdata.append(currentline)     
    currentline = ""

outputdata[0:3]

['PPY,1,1,1,1,1,1,1,\n']

### Export result as CSV file

In [26]:
output = ".\\PPY\\"
outCHAfilename = output+"pep CHA report"+".csv"  
outcfreqfilename = output+"pep cfreq report.csv"

print("The result will be stored in folder ["+output+"]")
print("="*12)
print("The compound's names and number of conformers will be expored in ["+outcfreqfilename+"]")
print("="*12)
print("The CHA data will be stored in  folder ["+outCHAfilename+"]")

The result will be stored in folder [.\PPY\]
The compound's names and number of conformers will be expored in [.\PPY\pep cfreq report.csv]
The CHA data will be stored in  folder [.\PPY\pep CHA report.csv]


In [27]:
with open (outCHAfilename,"w") as wf: #generate the file which used to calculate CHA score
    wf.write("cname\n")
    for cdata in outputdata:
        wf.write(cdata)
    wf.close()

#generate the all molecule and its frequence appeared in origin dataset
#This file will utilized in CCA score calculate
with open (outcfreqfilename,"w") as f:  
    f.write("cname,frequence\n")
    for data in cfrequence:
        f.write(data+","+str(cfrequence[data])+"\n")
    f.close()

print("Successfully!")

Successfully!
